# Data Inspection and Validation
**Successfully loading data and prepairing it for visualization**

NOTE: Goal is to interject how best to format data (dataframe, proper size etc) to take the most advantage of RAPIDS

## Overview and Requirements
Super short version of intro notebook and restate requirments

## Imports
Can this run on your system

## Loading Data
Can you actually download the data ( and can you share it )

## Data Shape
Type of data, size, rows / cols, number of values, sparsity, max/min


## Data Utility
What are useful cols, what do they mean in the real world, is it useful for my problem, do I need to suppliment the data (e.g. get boundary file etc)


## Inspection
Various visualization techniques to validate if this data makes sense ( e.g. map for proper geospatial encoding, line for time series...)

## Cleanup
Various techniques to clean up and formatting for RAPIDS viz

## Summary of Data